In [30]:
import pandas as pd
import dexpy
from dexpy import simplex_lattice
from dexpy.model import ModelOrder

factors = {'Triton  / %':0.1,'DMSO /%':4,'Arginine / mM':150,'BSA /mg/ml':0.1}

design = simplex_lattice.build_simplex_lattice(len(factors),model_order = ModelOrder.quadratic)
design.columns = factors

for i,j in zip(design,factors):
    design.loc[:,i] *= factors[i]

design.to_csv('ResponseSurfaceDesign.csv')
design

,Triton / %,DMSO /%,Arginine / mM,BSA /mg/ml
0.0,0.10,0.0,0.0,0.00
1.0,0.00,4.0,0.0,0.00
2.0,0.00,0.0,150.0,0.00
3.0,0.00,0.0,0.0,0.10
4.0,0.05,2.0,0.0,0.00
5.0,0.05,0.0,75.0,0.00
6.0,0.05,0.0,0.0,0.05
7.0,0.00,2.0,75.0,0.00
8.0,0.00,2.0,0.0,0.05
9.0,0.00,0.0,75.0,0.05


In [29]:
StockConcs = {'Triton  / %':10,'DMSO /%':100,'Arginine / mM':500,'BSA /mg/ml':10}

def V1(v2,C1,C2):
    return (C2*v2)/C1


def VolToAdd(RowFromExpDesign, StockConcs,TotalVol_ul):
    # columns in the design and keys in the stock concs need to 
    # be in the same order
    
    VolToAddCols = ['Triton  / ul','DMSO / ul','Arginine / ul','BSA / ul']
    
    output = pd.DataFrame([[V1(TotalVol_ul, StockConcs[j],i) for i,j in zip(RowFromExpDesign, StockConcs)]],\
                          columns = VolToAddCols)

    VolWater = TotalVol_ul-output.sum().sum()
    output['Water'] = VolWater
    return output



plan = pd.DataFrame([],columns = ['Triton  / ul','DMSO / ul','Arginine / ul','BSA / ul','Water'])

for i in design.index:
    temp = VolToAdd(design.loc[i,:],StockConcs, 5_000)
    plan=plan.append(temp)

    
plan = plan.reset_index(drop=True)
plan

,Triton / ul,DMSO / ul,Arginine / ul,BSA / ul,Water
0,50.0,0.0,0.0,0.0,4950.0
1,0.0,200.0,0.0,0.0,4800.0
2,0.0,0.0,1500.0,0.0,3500.0
3,0.0,0.0,0.0,50.0,4950.0
4,25.0,100.0,0.0,0.0,4875.0
5,25.0,0.0,750.0,0.0,4225.0
6,25.0,0.0,0.0,25.0,4950.0
7,0.0,100.0,750.0,0.0,4150.0
8,0.0,100.0,0.0,25.0,4875.0
9,0.0,0.0,750.0,25.0,4225.0
